In [2]:
import syft as sy
from syft.client.api import ServerIdentity
from syft.service.request.request import RequestStatus

In [6]:
# Establish a connection to each hospital server
northside = sy.connect(url='localhost', port=8096)   
northside_client = northside.login(
    email='gsingal3@gatech.edu',
    password='testing123'
)
emory = sy.connect(url='localhost', port=8095)
emory_client = emory.login(
    email='gsingal3@gatech.edu',
    password='testing123'
)
grady = sy.connect(url='localhost', port=8094)
grady_client = grady.login(
    email='gsingal3@gatech.edu',
    password='testing123'
)

Logged into <northside_hospital: High side Datasite> as <gsingal3@gatech.edu>
Logged into <grady_hospital: High side Datasite> as <gsingal3@gatech.edu>
Logged into <emory_hospital: High side Datasite> as <gsingal3@gatech.edu>


In [7]:
# Obtain the asset for the dataset from each of the client hospitals
northside_asset = northside_client.datasets.get_all()[0].assets[0]
emory_asset = emory_client.datasets.get_all()[0].assets[0]
grady_asset = grady_client.datasets.get_all()[0].assets[0]



In [9]:
# Embedding function to interface with the datasites
@sy.syft_function_single_use(data=northside_asset)
def create_embeddings(data):
    from sentence_transformers import SentenceTransformer
    class CustomEmbeddings:
        def __init__(self, model_name="all-MiniLM-L6-v2"):
            self.model = SentenceTransformer(model_name)

        def embed_documents(self, texts):
            """
            Embeds a list of documents.
            """
            if isinstance(texts, str):
                texts = [texts]
            return self.model.encode(texts, convert_to_numpy=True).tolist()

        def embed_query(self, query):
            """
            Embeds a single query.
            """
            return self.model.encode([query], convert_to_numpy=True).tolist()[0]
    texts = data.apply(lambda row: ', '.join(f"{col}: {val}" for col, val in row.items()), axis=1).tolist()
    embedding = CustomEmbeddings()
    vectors = embedding.embed_documents(texts=texts)
    return vectors

SyftSuccess: Syft function 'create_embeddings' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [11]:
northside_project = sy.Project(
    name='Embeddings of hospital information',
    description='Using the hospital information for the retrieval-augmented generation system',
    members=[northside_client]
)
emory_project = sy.Project(
    name='Embeddings of hospital information',
    description='Using the hospital information for the retrieval-augmented generation system',
    members=[emory_client]
)
grady_project = sy.Project(
    name='Embeddings of hospital information',
    description='Using the hospital information for the retrieval-augmented generation system',
    members=[grady_client]
)

northside_res = northside_project.create_code_request(create_embeddings, northside_client)
emory_res = emory_project.create_code_request(create_embeddings, emory_client)
grady_res = grady_project.create_code_request(create_embeddings, grady_client)

northside_res.send()
emory_res.send()
grady_res.send()

In [ ]:
northside_ptr = northside_client.code.create_embeddings(data=northside_asset)
emory_ptr = emory_client.code.create_embeddings(data=emory_asset)
grady_ptr = grady_client.code.create_embeddings(data=grady_asset)



NameError: name 'asset' is not defined